# Tutorial 4 - Data Simulation


*Written and revised by Jozsef Arato, Mengfan Zhang, Dominik Pegler*  
Computational Cognition Course, University of Vienna  
https://github.com/univiemops/tewa1-computational-cognition

---

## This week's lab:

We will introduce you to data simulation, which is a valuable tool in psychological research that can be used for various scenarios. For example, before conducting the actual experiment, you can run simulations with different conditions and variables to evaluate the design of your experiments. It is also very common to manipulate parameters and explore possible situations during the simulations to understand potential outcomes. Simulations also allow you to estimate statistical power under different conditions, helping you to optimize sample sizes to ensure the validity and reliability of your studies. We will show you how to do this with several examples in this notebook. 

**Learning goals:**  
When finishing this tutorial, you should be able to ...
* simulate different conditions systematically for experiment design evaluation and power analysis
* perform standard model recovery procedures and evluate your model

## Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg, stats

## 1. Simulation for hypothesis testing

In this example, we simulate a scenario in which the effect of a new drug on reaction time performance is compared to a placebo. Let's pretend that we have already done a lot of literature research and we hypothesize that the new drug is better than the placebo. We also expect that the new drug can improve reaction time by about 20ms, and we have the knowledge that the population mean and standard deviation for our experimental task is 520ms $\pm$ 100ms. (I know this may sound very arbitrary here. In real practice, we should carefully choose simulation parameters based on some rationale, often coming from the literature.)

Now let's simulate 2 groups of 20 participants each with the parameters we have assumed. Again, we assume normal distributions:

In [ ]:
# create some random data

new_drug = np.random.normal(loc=500, scale=100, size=20)
placebo = np.random.normal(loc=520, scale=100, size=20)


# Data visualization

plt.hist(new_drug, alpha=0.5, label="new drug", bins=10)
plt.hist(placebo, alpha=0.5, label="placebo", bins=10)
plt.xlabel("reaction time (ms)")
plt.ylabel("number of participants")
plt.legend()
plt.show()

Okay, with the simulated data, we can analyze it with the independent samples t-test to compare the mean of two groups to see what result we can get. 

In [ ]:
stats.ttest_ind(
    new_drug, placebo, alternative="less"
)  # one-sided, the mean reaction time of new drug group is less than that of the placebo group

Now you can see whether the simulation result supports the hypothesis that the new drug is better than the placebo or not. 

You may be getting the idea that a standard data simulation procedure for hypothesis testing works pretty much the same as conducting an empirical study. You first define the hypothesis, then select the simulation method and generate data (similar to collecting data for an experiment). Then you implement a statistical test and perform hypothesis testing. Without the effort and resources of running a real experiment, you still have a chance to know what might happen. Also, you can easily "rerun the experiment" as many times as you want to see the potential results:

In [ ]:
# We repeat the above procedures 1000 times, and store the p-value of each iteration's t-test

n_sim = 1000
p_values = np.zeros(n_sim)

for i in range(n_sim):
    # generate data
    new_drug = np.random.normal(500, 100, 20)
    placebo = np.random.normal(520, 100, 20)

    # statistical test
    p_values[i] = stats.ttest_ind(new_drug, placebo, alternative="less")[1]

p_sig_ratio = np.sum(p_values < 0.05) / n_sim  # 0.05 as significance level
print("Chance of getting a significant result:", p_sig_ratio)

In [ ]:
# Visualize the obtained p values with a histogram

plt.hist(p_values, bins=20)
plt.axvline(x=0.05, color="r")  # vertical line for significance level
plt.xlabel("p_values")
plt.ylabel("frequency")
plt.title("Distribution of simulated p values")
plt.show()

Although you can get a sense of the p-value without doing the actual experiment, it is important that you never use simulation for p-hacking, and they are inherently different. Simulation is done transparently with predefined hypotheses, models, and analysis procedures to explore theoretical concepts, design experiments, and improve our understanding of possible outcomes. P-hacking manipulates data analysis procedures to achieve desired results, often without transparency or pre-specification of hypotheses. Simulation can be a valuable tool only when used with scientific integrity.

## 2. Simulation for evaluating experimental design and power analysis

In the first example, we set all parameters to a fixed number. However, when you run the real experiment, many things can vary. For example, the average reaction time of your participants may be slightly different from the literature, the noise level in your data may be different from other people's data. Also, you have to decide how many participants to collect, rather than choosing an arbitrary number.

So, the first step we need to do is write a function that allows parameters to be changed. Define a function called "exp_eval" that takes 5 input variables in order:
1. the mean of the new_drug group 
2. the mean of the placebo group
3. the standard deviation for both groups (Since homogeneity of variance is one assumption of t-test, we assume the two groups have the same standard deviation.)
4. the number of participants, also equal for both groups
5. the number of simulations 

The function should first simulate normally distributed data for two groups using the input variables, and then calculate the p-value using an independent samples t-test (one-tailed, new_drug mean < placebo mean). Repeat this procedure for the number of simulations and return the proportion of significant tests (p < 0.05) and all p-values.

In [ ]:
def exp_eval(# YOUR CODE HERE):
   # YOUR CODE HERE
    return sig_ratio, p_values

In [ ]:
# We set all the parameters the same as in the first example.
# Run this cell and you should get a plot similar to the one above if your function is correct.
sig_ratio, p_values = exp_eval(500, 520, 100, 20, 1000)

plt.hist(p_values, bins=20)
plt.axvline(x=0.05, color="r")  # vertical line for significance level
plt.xlabel("p_values")
plt.ylabel("frequency")
plt.title("Distribution of simulated p values")
plt.show()

With the function you just wrote, you can now systematically change the parameters to evaluate how different conditions can change the results of the experiment. Let's assume that different doses of the new drug have a linear effect on reaction time, and that the doses we plan to give the participants will ideally result in 410, 420, ... 490, 500 ms reaction time. So we change the mean reaction time of the new drug group and keep the other parameters fixed:

In [ ]:
conds_new_drug = 10
mean_new_drug = np.linspace(start=410, stop=500, num=conds_new_drug)


# For each new drug group mean, run simulation

powers = np.zeros(conds_new_drug)

for i in range(conds_new_drug):
    powers[i], _ = exp_eval(
        mean_new_drug[i], 520, 100, 20, 1000
    )  # "_" as a variable name for the second output of the function, which we ignore

print(powers)

In [ ]:
# Visualize the simulation results under different conditions

plt.plot(mean_new_drug, powers, marker="o")
plt.xlabel("New drug group mean (ms)")
plt.ylabel("Power")
plt.xticks(np.linspace(410, 500, 10))
plt.yticks(np.linspace(0, 1, 11))
plt.show()

Do you understand why we call the ratio of significant results as "Power" here? Check out the definition of *statistical power*!

Now, try it by yourself, systematically change the number of participants from 16 to 88 with interval of 8. Then visualize the results with a nice plot. We already set up other parameters you need for the "exp_eval" function below.

In [ ]:
mean_new_drug = 480
mean_placebo = 520
sd = 80
n_sim = 1000

# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE FOR DATA VISUALIZATION

If you did the above task correctly, you should have a power greater than 0.8 for conditions with more than 56 participants. 
You can also play around with different parameters to see how power changes accordingly. The difference between two group means is related to the effect size, and the standard deviation is related to the noise level in the data when performing a traditional power analysis. 

## 3. Simulation for performing model recovery 

In the field of computational cognition, it is common for researchers to propose some models/algorithms to explain or predict human behavior and mental processing. Of course, they would be interested to know how well their models are able to capture the underlying mechanism of the phenomenon. Model recovery is a procedure that can be helpful here. It assesses whether a computational model can accurately recover known parameters from simulated or observed data. By following the model recovery procedure, researchers can gain insight into the reliability and validity of their computational models and identify limitations or biases in parameter estimation methods.

Let's walk through a standard model recovery using a linear regression model. Imagine that you are a researcher working on infant growth (yes, you need to be very imaginative for this course). You develop a linear model to predict the height of infants as they grow up to 36 weeks:

In [ ]:
# Step 1: Define the model

b0 = 50  # cm, starting height of infants
b1 = 0.6  # growing rate: 0.6 cm/week

# You define your model as
# y = b0 + b1 * x

Now you need to generate simulated data using the defined model with known parameter values. Below we generate 40 infant heights from 0 to 36 weeks. Of course, in the real world there is always some noise in the data, so we need to generate some normally distributed noise as well.

In [ ]:
# Step 2: Generate simulated data and add noise to it

np.random.seed(221)
x_data = np.random.randint(0, 37, 40)  # weeks of infant
noise = np.random.normal(0, 8, 40)  # mean and sd of the noise is 0 and 8 cm
y_data = b0 + b1 * x_data + noise

We'd better make a plot to get an idea of what the simulated data looks like.

In [ ]:
# Visulization of the simulated data

plt.scatter(x_data, y_data)
plt.xlabel("Age (weeks)")
plt.ylabel("Heigth (cm)")
plt.show()

Now, we'll use the least squares method to fit a linear regression to the simulated data. We want to know if we can recover the known parameters.

Before we continue with the code, let's review some simple matrix algebra. When we run linear regression in Python, we typically store all of our independent variables (also called *predictors* or *regressors*) in a single matrix, which is called the **design matrix**. We call it that because it "designs" how we want to model our dependent variable. Python provides many different functions for fitting a linear regression. These functions often assume that the design matrix **X** has the shape of *N*(observations) * *P*(predictors), and the dependent variable **Y** has the shape of *N*, (or *N* * 1). Then the linear regression formula Y = $\beta_0 + \beta_1x_1 + \beta_2x_2 +···+ \beta_px_p + \epsilon$ can be concisely expressed using matrix algebra as $$Y = X\beta + \epsilon$$


In our case, we have 40 observations and 2 predictors, which are the intercept and the slope of the regression line. To model the intercept, we need to add a column of ones (do you understand why ones?) to "x_data". 

In [ ]:
x_data_with_intcp = np.column_stack((np.ones(40), x_data))
print("The first 10 rows of the design matrix:", "\n", x_data_with_intcp[:10, :], "\n")
print("The shape of the design matrix:", x_data_with_intcp.shape)

Now we have 2 predictors in the design matrix, we can fit the linear regression ($y_i = \beta_0 * 1+ \beta_1 * x_i + \epsilon$) using the least squares method:

In [ ]:
# Step 3: Fit model to simulated data

from scipy import linalg

betas = linalg.lstsq(x_data_with_intcp, y_data)[0]
print("Estimated b0 and b1:", betas)

Hmmm, our estimated parameters do not look too different from our known (true) parameters. To quantitatively evaluate the accuracy of the parameter recovery, we calculate the correlation and the root mean square error between the estimated y and the original "y_data". 

In [ ]:
# Step 4: Calculate the recovery metrics to evaluate the accuracy of the parameter recovery

y_est = x_data_with_intcp.dot(
    betas
)  # np.dot() returns dot product of two arrays, check out how it differnt from np.multiply()
correlation = np.corrcoef(y_est, y_data)[0, 1]
rmse = np.sqrt(np.mean((y_est - y_data) ** 2))

print("True Intercept:", b0)
print("Estimated Intercept:", betas[0])
print("True Slope:", b1)
print("Estimated Slope:", betas[1])
print("Correlation:", correlation)
print("RMSE:", rmse)

You can plot the resulting regression line along with the linear model you defined for comparison.

In [ ]:
# Step 5: Visualize the fitted model and the defined model

plt.scatter(x_data, y_data)
plt.plot(x_data, b0 + x_data * b1, "g--", label="defined model")
plt.plot(x_data, y_est, "k:", alpha=0.5, label="fitted model")
plt.xlabel("Age (weeks)")
plt.ylabel("Heigth (cm)")
plt.legend()
plt.show()

Alright, we just showed you a simple example of model recovery using a linear regression model. You can extend this approach to more complex models and datasets by adjusting the model definition, simulated data generation, and parameter estimation procedure.

Now, you may ask, how can this procedure be used to help you evaluate your model? One answer is that you can systematically simulate different conditions and run model recovery for each condition to see your model's performance. For example, as the researcher proposing the infant growth model, you are now interested in knowing two aspects of the model before selling it to academia: 1. how much noise in the data your model can tolerate; 2. how much data you need to recover the model parameters. To do this, you need to simulate different standard deviation values for the noise and different numbers of infants.

First, let's functionalize the above procedures since we'll need to repeat them many times. Write a function called "simulator" that takes "n_sub" and "noise_sd" as inputs and "betas" from the fitted linear model as outputs in the cell below. We will only use step 2 and step 3 for now. Try to write the code yourself, not just copy and paste the code above.

In [ ]:
def simulator(n_sub, noise_sd):
    # Step 2: Generate simulated data and add noise to it
    # YOUR CODE HERE

    # Step 3: Fit model to simulated data
    # YOUR CODE HERE

    return betas

Then we can use the function to go through all combinations of different noise levels and number of infants to see how the model performs under those conditions. For each condition, we run the simulation and model recovery 100 times and save the average performance to avoid randomness/bias.

In [ ]:
n_sub = np.array([20, 40, 80])  # number of infants
noise_sd = np.array([5, 10, 20, 40])  # sd of the noise distribution
outs = np.zeros((12, 2))  # variable to store recovered betas from all coditions
row_counter = 0

for i in n_sub:
    for j in noise_sd:
        this_cond = np.zeros(
            (100, 2)
        )  # for each condition, run the simulation 100 times
        for k in range(100):
            this_cond[k, :] = simulator(i, j)
        outs[row_counter, :] = this_cond.mean(
            axis=0
        )  # save the mean betas from 100 simulations
        row_counter += 1

print(outs)

In [ ]:
# Data visulization

x = np.linspace(0, 36, 100)
plt.figure(figsize=(6, 18))

for i in range(1, 4):
    plt.subplot(3, 1, i)
    plt.plot(x, 50 + x * 0.6, "k--", label="defined model")
    plt.plot(
        x, outs[(i - 1) * 4, 0] + x * outs[(i - 1) * 4, 1], "b:", label="5 noise sd"
    )
    plt.plot(
        x,
        outs[(i - 1) * 4 + 1, 0] + x * outs[(i - 1) * 4 + 1, 1],
        "r:",
        label="10 noise sd",
    )
    plt.plot(
        x,
        outs[(i - 1) * 4 + 2, 0] + x * outs[(i - 1) * 4 + 2, 1],
        "y:",
        label="20 noise sd",
    )
    plt.plot(
        x,
        outs[(i - 1) * 4 + 3, 0] + x * outs[(i - 1) * 4 + 3, 1],
        "g:",
        label="40 noise sd",
    )
    plt.xlabel("Age (weeks)")
    plt.ylabel("Heigth (cm)")
    plt.legend()
    plt.title(str(n_sub[i - 1]) + " " + "infants")

You can run the above code multiple times, and even change the data simulation procedures, to assess how well the model recovers from perturbations (small samples, noise, outliers, missing data, etc.). How effectively your model recovers and produces accurate results can indicate its stability. You can also explore how your model behaves during recovery to identify its failure modes and limitations, and try to improve it accordingly. 

## Exercise 1: Simulation with mean difference

1. Choose 10 different standard deviations (e.g., ranging from 2 to 210) and 8 different group sizes (e.g., also ranging from 2 to 210) (Hint: you can use the argument `dtype=int` in `np.linspace()` to create integer values).
2. Simulate a number (e.g., 100) of $t$-tests for new random data created from combinations of above group sizes and standard deviations (keep the mean difference fixed, e.g., ` mean_1 = 500`  and `mean_2 = 520` )
3. Store the proportion of significant $p$-values (e.g., $p$ < 0.05) obtained from the 100 $t$-test simulations for each combination in a 2D numpy array. Each value (proportion) in the array corresponds to a particular standard deviation (row) and group size (column).
4. Use `plt.pcolor()` to visualize the resulting array, representing the proportions. Additionally, add x and y labels to denote the two parameters: standard deviation and group size.

**Question**: What do you observe and why?

In [ ]:
#
#
#

## Exercise 2: Simulation with no mean difference


Make a similar systematic simulation, but with no mean difference (e.g., ineffective drug) and calculate the propotion of significant tests as you manipulate the standard deviation and the group size.

**Question**: What do you observe and why could that be the case?


In [ ]:
#
#
#

When you are finished with the figure, compare it to the previous figure from the true difference simulation.

**Question**: What do you observe and why?